If you have not used gammapy before, you'll want to follow this link https://docs.gammapy.org/1.1/getting-started/install.html#installation to install the package first. Even if you will not be working with Gammapy catalogs with this analysis, it could be useful to go ahead and downoal the tutorial materials in this link https://docs.gammapy.org/dev/getting-started/index.html 

This notebook adapts the gammapy multi instrument joint 3D and 1D analysis to the context of a vegas fits file.

In [1]:
import gammapy

In [3]:
import astropy
from astropy.io import fits

In [2]:
from pathlib import Path
from astropy import units as u
import matplotlib.pyplot as plt
from gammapy.datasets import Datasets, FluxPointsDataset, SpectrumDatasetOnOff
from gammapy.estimators import FluxPoints, FluxPointsEstimator
from gammapy.maps import MapAxis
from gammapy.modeling import Fit
from gammapy.modeling.models import Models

/nevis/milne/files/cmm2399/.conda/envs/gammapy-1.1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#We'll need to edit some headers in order for gammapy to read of files, this will let us see the extension (1 SpectrumTable) we need to edit
fits_image_filename = astropy.io.fits.open('/path/to/your.fits')
fits.info('/path/to/your.fits')

In [ ]:
print("Before modifications:")
print()
print("Extension 1:")
print(repr(fits.getheader('/path/to/your.fits', 1)))

You should look at the SED specifications for the headers and make sure to modify the right headers in accordance, but the changes I specify should work.

In [ ]:
with fits.open('/path/to/your.fits', 'update') as f:
    for hdu in f:
        hdu.header['TTYPE3'] = 'e_min'
        hdu.header['TTYPE4'] = 'e_max'
        hdu.header['TTYPE5'] = 'dnde'
        hdu.header['TTYPE6'] = 'dnde_err'
        hdu.header['TTYPE7'] = 'dnde_ul'
        
print("After modifications:")
print()
print("Extension 1:")
print(repr(fits.getheader('/path/to/your.fits', 1)))

Remember to repeat the previous steps with all the fits files you plan on using. You will probably need to do some more edits if your file looks does not look like the example hawc file in this tutorial https://docs.gammapy.org/1.1/tutorials/analysis-3d/analysis_mwl.html#hawc-1d-dataset-for-flux-point-fitting, which you should have (if you downloaded the tutorial dataset) at ./gammapy-data/1.1/hawc_crab/HAWC19_flux_points.fits

you can open it using 
    fits_image_filename = astropy.io.fits.open()
    fits.info()
and pring the first extension (a table) with
    print("Before modifications:")
    print()
    print("Extension 1:")
    print(repr(fits.getheader('./gammapy-data/1.1/hawc_crab/HAWC19_flux_points.fits', 1)))

Notice it has TS and is_UL columns, which the Vegas files do not. The following steps show how you can emulate this structure.

In [ ]:
from astropy.table import Table
data = astropy.io.fits.open('/path/to/your.fits', mode='update', extname='SpectrumTable')
table = Table.read('/path/to/your.fits', format='fits')

In [ ]:
table #Do this if you don't know how many rows of data your extension 1 table has

In [ ]:
#This will calculate the TS 
import math
sigval2 = []
boolean = []
for i in range(22):
    x = table['Significance'][i] 
    y = x**2
    sigvals.append(y)
    if y < 4:
        boolean.append(True)
    else:
        boolean.append(False)
    #print(y)
print(sigvals)
print(boolean)

In [ ]:
#make sure 12 and 13 are the right column positions for you. this will save the editted table to a new fits file, but if you wish to simply overwrite your previous table, use the same path as before and add overwrite=True.
col12 = astropy.table.Column(name= 'TS', data=sigval2)
col13 = astropy.table.Column(name='is_ul', data=boolean)
table.add_column(col12)
table.add_column(col13)
table.write('/path/to/your/new.fits', format='fits')

In [ ]:
table #now you can see that the new columns were created

Repeat these steps for your other fits file:

In [ ]:
from astropy.table import Table
data = astropy.io.fits.open('/path/to/your/other.fits', mode='update', extname='SpectrumTable')
table = Table.read('/path/to/your/other.fits', format='fits')

In [ ]:
table #check number of rows

In [ ]:
#This will calculate the TS 
import math
sigval2 = []
boolean = []
for i in range(22):
    x = table['Significance'][i] 
    y = x**2
    sigvals.append(y)
    if y < 4:
        boolean.append(True)
    else:
        boolean.append(False)
    #print(y)
print(sigvals)
print(boolean)

In [ ]:
#make sure 12 and 13 are the right column positions for you. this will save the editted table to a new fits file, but if you wish to simply overwrite your previous table, use the same path as before and add overwrite=True.
col12 = astropy.table.Column(name= 'TS', data=sigval2)
col13 = astropy.table.Column(name='is_ul', data=boolean)
table.add_column(col12)
table.add_column(col13)
table.write('/path/to/your/other/new.fits', format='fits')

In [ ]:
table #now you can see that the new columns were created

Now we can call the new files and actually create the plot!

In [ ]:
filename = "/path/to/your/new.fits"
flux_points_yourfile = FluxPoints.read(filename, hdu=1, sed_type="dnde")

dataset = FluxPointsDataset(data=flux_points_yourfile, name="yourfile")

print(dataset)

In [ ]:
flux_points_yourfile.to_table() #check everything you need to be read is... there

In [ ]:
#check things for your second file
filename = "/path/to/your/other/new.fits"
flux_points_yourotherfile = FluxPoints.read(filename, hdu=1, sed_type="dnde")

dataset = FluxPointsDataset(data=flux_points_yourotherfile, name="yourotherfile")

print(dataset)

Before moving on to creating the plot, change all of the dnde and dnde_err cells that are upper limits (True) to Null and all the dnde_ul cells that are not an upper limit (False) to Null. You can use the table.replace() command and manually do this, or create a boolean using is_UL. I did this manually with fv in my terminal.

In [ ]:
#Now everything should be working well, and you can create your plot!
fig, ax = plt.subplots(figsize=(8, 6))

energy_bounds = [0.1, 100] * u.TeV
sed_type = "dnde"

flux_points_Martinez.plot(ax=ax, sed_type=sed_type, label="yourfile")
flux_points_Drake.plot(ax=ax, sed_type=sed_type, label="yourotherfile")

ax.set_xlim(energy_bounds)
ax.legend()
plt.show()